In [1]:
import numpy as np
import scipy as sp
import tensorflow
import keras
#from keras.layers import Dense, Sequential, 
from Chess_Version_4 import Game, Custom_Game, Player, Board, square, Piece, Pawn, Queen, Rook, King, Knight, Bishop
import pandas as pd
from multiprocessing import Process
from dask.distributed import LocalCluster, Client
import os
import dask.dataframe as dd
print('Number of CPUs in the system: {}'.format(os.cpu_count()))

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
df = pd.read_csv('chessData.csv')[:10000000]
ddf = dd.from_pandas(df, npartitions=6)

In [3]:
ddf.Evaluation.head(100)

0     -10
1     +56
2      -9
3     +52
4     -26
     ... 
95     -6
96    -49
97      0
98    -36
99      0
Name: Evaluation, Length: 100, dtype: object

In [5]:
#client.close()
#client.shutdown()
#!pip install "dask[distributed]" 

#client = Client(n_workers=6)
#cluster = LocalCluster()

In [13]:
def FEN_to_Dat(fen):
    
    pieces , curr_play, castling = fen.split(" ")[:3]
    
    cp = {'b':0,'w':1}
    
    cnts = {'P':0,'N':0,'B':0,'R':0,'Q':0,'K':0,'p':0,'n':0,'b':0,'r':0,'q':0,'k':0}
    
    for i in pieces:
        if i.isalpha():
            cnts[i] += 1
            
    rows = pieces.split('/')
    if rows[-1] == '':
        rows = rows[:-1]
    rows = rows[::-1]
    
    cg = Custom_Game()
    
    for i in range(len(rows)):
        if len(rows[i]) == 1 and rows[i] == '8':
            rows[i] = '********'
        string = ''
        for j in range(len(rows[i])):
            if rows[i][j].isnumeric():
                string += int(rows[i][j])*'*'
            else:
                string += rows[i][j]
                    
        rows[i] = string
    board = [[i for i in r] for r in rows]
    
    p_dict = {'p': Pawn,'n':Knight,'b':Bishop,'r':Rook,'q':Queen,'k':King}
    
    for j in range(8):
        for i in range(8):
            name = board[j][i]
            if board[j][i] == 'k':
                k = p_dict[name.lower()](cg.ChessBoard.squares[j][i],'Black',name,cg.ChessBoard)
                if ('k' or 'q') not in castling:
                    k.move_count = 1
            elif board[j][i] == 'K':
                K = p_dict[name.lower()](cg.ChessBoard.squares[j][i],'White',name,cg.ChessBoard)
                if ('K' or 'Q') not in castling:
                    K.move_count = 1
                
    cg.ChessBoard.add_pieces([k,K])
    
    kings = {'White':K,'Black':k}
    
    p_list = []
    
    for c in range(8):
        for r in range(8):
            if board[c][r] != '*' and board[c][r][0].lower() != 'k':
                
                mcnt = 1
                name = board[c][r]
                
                if name.lower() == 'p':
                    if name == 'P' and c == 1:
                        mcnt = 0
                    elif name == 'p' and c == 6:
                        mcnt = 0
                elif name.lower() == 'r':
                    
                    if name == 'R':
                        if (r,c) == (0,0) and 'Q' in castling:
                            mcnt = 0
                        elif (r,c) == (7,0) and 'K' in castling:
                            mcnt = 0
                    elif name == 'r':
                        if ((r,c) == (0,7) and 'q' in castling) or ((r,c) == (7,7) and 'k' in castling):
                            mcnt = 0
                    
                name = name+str(cnts[name])
                cnts[name[0]] -= 1
                    
                if name[0].lower() != name[0]:
                    col = 'White'
                elif name[0].lower() == name[0]:
                    col = 'Black'

                p = p_dict[name[0].lower()](cg.ChessBoard.squares[c][r],col,name,cg.ChessBoard,kings)
                p.move_count = mcnt
                p_list.append(p)
                
    cg.ChessBoard.add_pieces(p_list)
    b = Player('Butt','Black',cg.ChessBoard)
    w = Player('Deriere','White',cg.ChessBoard)
    playerdict = {'w':w,'b':b}
    cg.WhitePlayer,cg.BlackPlayer = w,b

    cg.ChessBoard.heat_setup()
    cg.ChessBoard.en_passant_setup()
    cg.ChessBoard.players = {'White':w,'Black':b}
    cg.ChessBoard.current_player = playerdict[curr_play]
    
    heats = cg.ChessBoard.heatmap()
    
    piece_cnts = {'P':8*[8*[0]],'N':8*[8*[0]],'B':8*[8*[0]],'R':8*[8*[0]],'Q':8*[8*[0]],'K':8*[8*[0]]}
    
    cas = [[0,0],[0,0]]
    
    for i in castling:
        
        if i == 'K':
            cas[0][0] = 1
        elif i == 'Q':
            cas[0][1] = 1
        elif i == 'k':
            cas[1][0] = 1
        elif i == 'q':
            cas[1][1] = 1
    
    for col in cg.ChessBoard.pieces:
        for piece in cg.ChessBoard.pieces[col]:
            
            x,y = cg.ChessBoard.pieces[col][piece].position
            p = piece[0]
            
            b = piece_cnts[p.upper()]
            a = b[y].copy()
            val = 1 if col == 'White' else -1
            a[x] = val
            b[y] = a
            piece_cnts[p.upper()] = b
            
    h_arr = []  
    for i in heats:
        h_arr += [heats[i]]
            
    p_arr = []
    for i in piece_cnts:
        p_arr += [piece_cnts[i]]
    
    
    b = w = None
    kings = None
    cg.ChessBoard = None
    cg = None
    #heats = None
    piece_cnts = None
    cnts = None
    board = None
    p_list = None
    pieces = castling = None
    p_dict = None
    p = None
    
    return h_arr,p_arr,cas,cp[curr_play]
    
    
    
a = FEN_to_Dat('rnbqkbnr/pppppppp/8/8/8/P7/1PPPPPPP/RNBQKBNR/ b KQkq - 1 0')

In [56]:
#'rnbqkbnr/pppppppp/8/8/8/P7/1PPPPPPP/RNBQKBNR/ b KQkq - 1 0'.split(' ')[0].split('/')[-1] == ''

-1

In [18]:
def test_train_split(data,ratio = 0.8):
    train_a = ratio
    
    N = len(data)
    
    trainheat,testheat = np.array([]),np.array([])
    trainpiece,testpiece = np.array([]),np.array([])
    traincas,testcas = np.array([]),np.array([])
    traincp,testcp = np.array([]),np.array([])
    trainy,testy = np.array([]),np.array([])
    for n in range(N):
        
        try:
            
            h,p,c,cp = FEN_to_Dat(data.FEN[n])

            if n < int(train_a*N):

                np.append(trainheat,[h])
                np.append(trainpiece,[p])
                np.append(traincas,[c])
                np.append(traincp,[cp])
                np.append(trainy,[data.Evaluation[n]])
                
                
            elif n >= int(train_a*N):

                np.append(testheat,[h])
                np.append(testpiece,[p])
                np.append(testcas,[c])
                np.append(testcp,[cp])
                np.append(testy,[data.Evaluation[n]])
        except:
            
            print('Uh oh')
    
    train = pd.DataFrame(zip(trainheat,trainpiece,traincas,traincp,trainy), columns = ['Heats','Pieces','Castle','Curr_Play','Value'])
    test  = pd.DataFrame(zip(testheat,testpiece,testcas,testcp,testy), columns = ['Heats','Pieces','Castle','Curr_Play','Value'])
    
    trainheat = trainpiece = traincas = traincp = trainy = None
    testheat = testpiece = testcas = testcp = testy = None
    
    print('Success!')
    return train,test

In [14]:
def test_train_split(data,ratio = 0.8):
    train_a = ratio
    
    N = len(data)
    
    trainheat,testheat = np.array([]),np.array([])
    trainpiece,testpiece = np.array([]),np.array([])
    traincas,testcas = np.array([]),np.array([])
    traincp,testcp = np.array([]),np.array([])
    trainy,testy = np.array([]),np.array([])
    for n in range(N):
        
        try:
            
            h,p,c,cp = FEN_to_Dat(data.FEN[n])
            
            string = data.Evaluation[n]
            
            if '#' in data.Evaluation[n]:
                
                string = data.Evaluation[n].replace('#','')
                
            val = int(string)
            
            yield (np.array([h,p]),np.array(c),np.array(cp),np.array(val))
        
        except:
            
            print(data.FEN[n])
        
def generate(df,labels,batch_size,idx):
    
    xvar,yvar = labels
    batch_hp = np.zeros((batch_size, 12, 8, 8))
    batch_c = np.zeros((batch_size,2,2))
    batch_cp = np.zeros((batch_size,))
    batch_val = np.zeros((batch_size,))
    
    while True:
        for i in range(batch_size):
            
            try:
                
                h,p,c,cp = FEN_to_Dat(df[xvar][idx])
            
                string = df[yvar][idx]
            
                if '#' in df[yvar][idx]:
                
                    string = df[yvar][idx].replace('#','')
                
                val = int(string)
                
                batch_hp[i] = np.array(h+p)
                batch_c[i] = np.array([c])
                batch_cp[i] = cp
                batch_val[i] = val
        
            except:
            
                print(df[xvar][idx])
                
            idx += 1
    
        yield [batch_hp,batch_c,batch_cp],batch_val

In [18]:
#result = ddf.map_partitions(test_train_split,ratio = 0.8)
#result.compute()
res = generate(df,['FEN','Evaluation'],32,0)
next(res)

([array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.],
           [ 1.,  2.,  2., ...,  1.,  2.,  1.],
           ...,
           [-1., -2., -2., ..., -2., -2., -1.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
  
          [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.],
           [ 1.,  0.,  1., ...,  1.,  0.,  1.],
           ...,
           [-1.,  0., -1., ..., -1.,  0., -1.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
  
          [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.],
           ...,
           [ 1.,  0.,  0., ...,  0.,  0.,  0.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.],
           [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
  
          ...,
  
          [[ 1.,  0.,  0., ...,  0.,  0.,  1.],
          

In [ ]:
from keras.models import Input
# Model

class Model:
    
    def __init__(self,df,shapes,batchsize,dropouts,activations,loss,metrics,epochs,gens):
        
        self.shapes = shapes
        self.batch = batchsize
        self.dor = dropouts
        self.act = activations
        self.loss = loss
        self.metrics = metrics
        self.epochs = epochs
        self.gens = gens
        
        hp = Input(shape = self.shapes[0])
        c = Input(shape = self.shapes[1])
        cp = Input(shape = self.shapes[2])
        
        hotpiece = Sequential(hp)
        cas = Sequential(c)
        
        c_p = Sequential(cp)
        
        
        
        
        comb = Keras.Layers.concatenate([hotpiece.output,cas.output,c_p.output])
        fin = Dense(32,activation = self.act['fin'][0])(comb)
        fin = Dropout(rate = self.dor)(fin)
        fin = Dense(1,activation = self.act['fin'][1])(fin)
        
        self.model = Model(inputs = [hotpiece.inputs,cas.inputs,cp.inputs],outputs = fin)
        
        self.model.compile(optimizer=ko.Adam(lr=0.0001), loss=self.loss, metrics=self.metrics)
        
    def summarize(self):
        
        return self.model.summary()
    
    def fit(self,df,range_idx_train,range_idx_val,train_steps,test_steps):
        
        self.model.fit_generator(generator = generator(df,['FEN','Evaluation'],self.batch,range_idx_train),
                                steps_per_epoch = train_steps,
                                validation_data = generator(df,['FEN','Evaluation'],self.batch,range_idx_test),
                                epochs = self.epochs, 
                                use_multiprocessing = True, workers = 6)

In [ ]:
NN_MK1 = Model

In [ ]:
# Performance Evaluation
# Validation

model.compile(optimizer=ko.Adam(lr=0.0001), loss='categorical_crossentropy', metrics='accuracy')

model.evaluate_generator(generator=validation_generator(validation_features, BATCH))

# Save the Model and Labels
model.save('Model.h5')